# PMG-2: 2D to 3D Reconstruction Pipeline

This notebook demonstrates the complete pipeline for converting 2D images to 3D models using depth estimation and 3D reconstruction techniques.

## Pipeline Overview
1. **Depth Estimation**: Using MiDaS or other depth estimation models
2. **Point Cloud Generation**: Converting depth maps to 3D point clouds
3. **3D Model Reconstruction**: Building mesh models from point clouds
4. **Visualization**: Displaying results in 3D space

In [ ]:
# Import required libraries
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import open3d as o3d

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

## Step 1: Load and Preprocess Input Image

In [ ]:
# Load input image
image_path = 'input_image.jpg'
img = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Display input image
plt.figure(figsize=(10, 6))
plt.imshow(img_rgb)
plt.title('Input Image')
plt.axis('off')
plt.show()

print(f'Image shape: {img.shape}')

## Step 2: Depth Estimation Using MiDaS

In [ ]:
# Load MiDaS model
midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS')
midas.to(device)
midas.eval()

# Load transforms
midas_transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
transform = midas_transforms.default_transform

print('MiDaS model loaded successfully')

In [ ]:
# Prepare input
input_batch = transform(img_rgb).to(device)

# Perform depth estimation
with torch.no_grad():
    prediction = midas(input_batch)
    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img_rgb.shape[:2],
        mode='bicubic',
        align_corners=False
    ).squeeze()

depth_map = prediction.cpu().numpy()
print(f'Depth map shape: {depth_map.shape}')

In [ ]:
# Visualize depth map
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(depth_map, cmap='magma')
plt.title('Depth Map')
plt.colorbar()
plt.axis('off')

plt.tight_layout()
plt.show()

# Save depth map
cv2.imwrite('depth_output.jpg', (depth_map / depth_map.max() * 255).astype(np.uint8))
print('Depth map saved as depth_output.jpg')

## Step 3: Generate 3D Point Cloud

In [ ]:
def depth_to_point_cloud(depth, rgb, focal_length=1000):
    """Convert depth map to 3D point cloud"""
    height, width = depth.shape
    
    # Create meshgrid
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    
    # Calculate 3D coordinates
    z = depth
    x = (x - width / 2) * z / focal_length
    y = (y - height / 2) * z / focal_length
    
    # Stack coordinates
    points = np.stack([x, y, z], axis=-1)
    points = points.reshape(-1, 3)
    
    # Get colors
    colors = rgb.reshape(-1, 3) / 255.0
    
    return points, colors

# Generate point cloud
points, colors = depth_to_point_cloud(depth_map, img_rgb)
print(f'Generated {len(points)} points')

In [ ]:
# Create Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

# Remove outliers
pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

# Save point cloud
o3d.io.write_point_cloud('point_cloud.ply', pcd)
print('Point cloud saved as point_cloud.ply')
print(f'Final point cloud contains {len(pcd.points)} points')

## Step 4: Visualize 3D Point Cloud

In [ ]:
# Visualize point cloud
o3d.visualization.draw_geometries(
    [pcd],
    window_name='3D Point Cloud',
    width=800,
    height=600,
    left=50,
    top=50,
    point_show_normal=False
)

## Step 5: Surface Reconstruction (Optional)

In [ ]:
# Estimate normals
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
)

# Perform Poisson surface reconstruction
print('Performing surface reconstruction...')
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
    pcd, depth=9
)

# Save mesh
o3d.io.write_triangle_mesh('reconstructed_mesh.ply', mesh)
print('3D mesh saved as reconstructed_mesh.ply')
print(f'Mesh contains {len(mesh.vertices)} vertices and {len(mesh.triangles)} triangles')

In [ ]:
# Visualize mesh
o3d.visualization.draw_geometries(
    [mesh],
    window_name='Reconstructed 3D Mesh',
    width=800,
    height=600,
    left=50,
    top=50
)

## Summary

This notebook demonstrated:
- Depth estimation from 2D images using MiDaS
- Point cloud generation from depth maps
- 3D visualization using Open3D
- Surface reconstruction using Poisson reconstruction

Output files:
- `depth_output.jpg`: Depth map visualization
- `point_cloud.ply`: 3D point cloud
- `reconstructed_mesh.ply`: 3D reconstructed mesh